In [1]:
import os
import pandas as pd
import numpy as np
#import pysam
import seqc
import matplotlib.pyplot as plt
import magic

In [2]:
def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

In [3]:
path="/home/lf2684/notebooks/seqc_07_Unsorted" 
os.chdir(path)
cwd = os.getcwd()

def readFastaSeq(filename):
    sequences = []
    with open(filename) as fh: 
        while True:
            fh.readline() #ignore this for now since we are reading seqs and ids from two separate files
            seq = fh.readline().rstrip()
            #seq = seq.replace('>', '') #remove leading >

            if len(seq) ==0: 
                break
            sequences.append(seq)
        return sequences
    
def readFastaCellid(filename):
    cellid = []
    with open(filename) as fh: 
        while True:
            cid = fh.readline().rstrip()
            fh.readline()
            if len(cid) ==0: 
                break
            cellid.append(cid)
        return cellid

In [12]:
#Enter sample name, sequence , type and gene

sample = '12H007_unsorted'

seq_type = 'wt' #mutated or normal
gene = 'NPM1' #NPM1 or DNMT3A
sequence = 'CTCTGG' #'AAGATCTCTGGCAGTGGAGGAA'

#'CTCTGG' wt NPM1
#'CTCTGCATGG' NPM1 for 89
#'CTCTGTCTGG' NPM1 for 07, 10


#table2['CTCTGTCTGG'] = 0 #TCTG
#table2['CTCTGCATGG'] = 0 #08H089

# TGCCTCGCCAAGTGGCT mutated DNMT3A for 12H010 
# TGCCTCGCCAAGCAGCT mutated DNMT3A for 09H058
# TGCCTCGCCAAGCGGCT normal DNMT3A

# TGCCTCGCCAAGCAGCT Mutated DNMT3A for 10H072
# TGCCTCGCCAAGTGGCT Mutated DNMT3A for other samples but 15H044



type_file = '.fasta'
dir_data = './seqc_07_Unsorted'
out_ext = '.csv'

In [13]:
seqs = readFastaSeq('12H007_NPM1.fasta')
ids = readFastaCellid('12H007_NPM1.fasta')
cell_barcode = [i.split(':')[1] for i in ids] 
UMI = [i.split(':')[2] for i in ids] 

In [14]:
table = pd.DataFrame()
table['seq'] = seqs
table['id'] = ids
table['barcode'] = cell_barcode
table['UMI'] = UMI
print(len(table))
table.head(5)

129434


,seq,id,barcode,UMI
0,GGTTGCAGTGAGCCTAGATCACGCCACTGCAACCTCTGCCTCCTGG...,>:ATCTTCATCTTTACAC:CTCCTAGAAGAC:T;A00333:164:H...,ATCTTCATCTTTACAC,CTCCTAGAAGAC
1,GGTTGCAGTGAGCCTAGATCACGCCACTGCAACCTCTGCCTCCTGG...,>:ATCTTCATCTTTACAC:CTCCTAGAAGAC:G;A00333:164:H...,ATCTTCATCTTTACAC,CTCCTAGAAGAC
2,GGTTGCAGTGAGCCTAGATCACGCCACTGCAACCTCTGCCTCCTGG...,>:ATCTTCATCTTTACAC:CTCCTAGAAGAC:A;A00333:164:H...,ATCTTCATCTTTACAC,CTCCTAGAAGAC
3,TGCCTGTAATCCCAGCTGCTTGGGAGGCTGAGGCAGGAGAATTGCT...,>:AGTAGCTTCGCGTTTC:CCATTGTGTCAT:A;A00333:164:H...,AGTAGCTTCGCGTTTC,CCATTGTGTCAT
4,TTGCTAATTTGGAGACTGATTCAGTCCCCTTTTGGCCCCCAAGTTA...,>:CGGGTGTCAAACGTGG:ATTCTGGGCATT:T;A00333:164:H...,CGGGTGTCAAACGTGG,ATTCTGGGCATT


In [15]:
from seqc.sequence.encodings import DNA3Bit

for i in range(0, len(table)):
#    print(i)
    b = DNA3Bit.encode(table.loc[table.index[i],'barcode'])
    u = DNA3Bit.encode(table.loc[table.index[i],'UMI'])
    table.loc[table.index[i],'final_cellid'] = b 
    table.loc[table.index[i],'final_cellid_UMI'] = u 
print(table.shape)   
table.head(5)

(129434, 6)


,seq,id,barcode,UMI,final_cellid,final_cellid_UMI
0,GGTTGCAGTGAGCCTAGATCACGCCACTGCAACCTCTGCCTCCTGG...,>:ATCTTCATCTTTACAC:CTCCTAGAAGAC:T;A00333:164:H...,ATCTTCATCTTTACAC,CTCCTAGAAGAC,1.574691e+14,5.567433e+10
1,GGTTGCAGTGAGCCTAGATCACGCCACTGCAACCTCTGCCTCCTGG...,>:ATCTTCATCTTTACAC:CTCCTAGAAGAC:G;A00333:164:H...,ATCTTCATCTTTACAC,CTCCTAGAAGAC,1.574691e+14,5.567433e+10
2,GGTTGCAGTGAGCCTAGATCACGCCACTGCAACCTCTGCCTCCTGG...,>:ATCTTCATCTTTACAC:CTCCTAGAAGAC:A;A00333:164:H...,ATCTTCATCTTTACAC,CTCCTAGAAGAC,1.574691e+14,5.567433e+10
3,TGCCTGTAATCCCAGCTGCTTGGGAGGCTGAGGCAGGAGAATTGCT...,>:AGTAGCTTCGCGTTTC:CCATTGTGTCAT:A;A00333:164:H...,AGTAGCTTCGCGTTTC,CCATTGTGTCAT,1.647017e+14,5.857698e+10
4,TTGCTAATTTGGAGACTGATTCAGTCCCCTTTTGGCCCCCAAGTTA...,>:CGGGTGTCAAACGTGG:ATTCTGGGCATT:T;A00333:164:H...,CGGGTGTCAAACGTGG,ATTCTGGGCATT,2.362205e+14,3.809207e+10


In [16]:
#search for target sequence in list of sequences- mark as 1 if present, 0 otherwise
table2 = table.copy()

table2[sequence] = 0

some_list = list(table['seq'])
b = list()

for i in range (0, len(some_list)):
    #print(i)
    a = []
    if sequence in str(some_list[i]):
        a=1
    else:
        a=0
    table2.loc[i,sequence] = a

In [17]:
print(sum(table2[sequence]))
table2.head(5)

10760


,seq,id,barcode,UMI,final_cellid,final_cellid_UMI,CTCTGG
0,GGTTGCAGTGAGCCTAGATCACGCCACTGCAACCTCTGCCTCCTGG...,>:ATCTTCATCTTTACAC:CTCCTAGAAGAC:T;A00333:164:H...,ATCTTCATCTTTACAC,CTCCTAGAAGAC,1.574691e+14,5.567433e+10,0
1,GGTTGCAGTGAGCCTAGATCACGCCACTGCAACCTCTGCCTCCTGG...,>:ATCTTCATCTTTACAC:CTCCTAGAAGAC:G;A00333:164:H...,ATCTTCATCTTTACAC,CTCCTAGAAGAC,1.574691e+14,5.567433e+10,0
2,GGTTGCAGTGAGCCTAGATCACGCCACTGCAACCTCTGCCTCCTGG...,>:ATCTTCATCTTTACAC:CTCCTAGAAGAC:A;A00333:164:H...,ATCTTCATCTTTACAC,CTCCTAGAAGAC,1.574691e+14,5.567433e+10,0
3,TGCCTGTAATCCCAGCTGCTTGGGAGGCTGAGGCAGGAGAATTGCT...,>:AGTAGCTTCGCGTTTC:CCATTGTGTCAT:A;A00333:164:H...,AGTAGCTTCGCGTTTC,CCATTGTGTCAT,1.647017e+14,5.857698e+10,0
4,TTGCTAATTTGGAGACTGATTCAGTCCCCTTTTGGCCCCCAAGTTA...,>:CGGGTGTCAAACGTGG:ATTCTGGGCATT:T;A00333:164:H...,CGGGTGTCAAACGTGG,ATTCTGGGCATT,2.362205e+14,3.809207e+10,0


In [18]:
# RESTRICT TO CELLS WITH THE SEQUENCE

table3 = table2.loc[table2[sequence] ==1,:]
table3['group'] = sample

unique_cells = list(np.unique(list(table3['barcode'])))
unique_UMI = list(np.unique(list(table3['UMI'])))
cells = list(np.unique(list(table['barcode'])))

print('This is the number of cells',len(unique_cells))
print('this is the number of UMI1', len(unique_UMI))
print('This is the number of total cells with reads mapping to gene',len(cells))

table3.head(5)

This is the number of cells 3966
this is the number of UMI1 7699
This is the number of total cells with reads mapping to gene 11710


<ipython-input-18-332ccb4d41ff>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table3['group'] = sample


,seq,id,barcode,UMI,final_cellid,final_cellid_UMI,CTCTGG,group
6,AGCTGAAAAACAAAGTTCCGTAAGTCTGGCCGGTAACTACATCTTT...,>:CGGGTGTCAAACGTGG:ATTCTGGGCATT:T;A00333:164:H...,CGGGTGTCAAACGTGG,ATTCTGGGCATT,2.362205e+14,3.809207e+10,1,12H007_unsorted
7,GAATAAGCCGCAATTCACTCTCTCTGGCACCTGAACTTTGGGGTAA...,>:TCCAGGTACTCCGCGG:GCTCCCACCGAG:C;A00333:164:H...,TCCAGGTACTCCGCGG,GCTCCCACCGAG,1.355636e+14,4.990976e+10,1,12H007_unsorted
9,CTCTCTGGCTCATTCGCAGCCGGCTAACCCGCCATATCTTACACAG...,>:GTCCACTGTCTCGGGT:GGTAAGATAGCA:G;A00333:164:H...,GTCCACTGTCTCGGGT,GGTAAGATAGCA,1.928681e+14,4.879799e+10,1,12H007_unsorted
10,CTCTCTGGCTCATTCGCAGCCGGCTAACCCGCCATATCTTACACAG...,>:GTCCACTGTCTCGGGT:GGTAAGATAGCA:A;A00333:164:H...,GTCCACTGTCTCGGGT,GGTAAGATAGCA,1.928681e+14,4.879799e+10,1,12H007_unsorted
14,CGCTCGCGAGATCTTCAGGGTCTATATATAAGCGCGGGGAGCCTGC...,>:CAAGAGGAGGTACAGC:GCCAACACCTAC:A;A00333:164:H...,CAAGAGGAGGTACAGC,GCCAACACCTAC,2.312815e+14,5.027466e+10,1,12H007_unsorted


In [19]:

table3.to_csv('./12H007_Unsorted_NPM1wt.csv')
#print(dir_data + sample + gene + seq_type + out_ext)

## stop here the rest is just me checking random stuff

In [22]:
npm1wt=table3
npm1wt.shape

(39, 8)

In [56]:
npm1muts=table3
npm1muts.shape

(7, 8)

In [18]:
#checking for duplicate cells between mut and wt

mut_ids = list(npm1muts['final_cellid'])
wt_ids = list(npm1wt['final_cellid'])

#ids for the wt cells also found in mut list
duplicates=[]
for i in wt_ids:
    if i in mut_ids:
        duplicates.append(i)
print(len(duplicates))

2351


In [19]:
print(len(np.unique(duplicates)))

961


In [20]:
unique=[] #unique cellids
counts=[] #counts of how many times each id appears (I THINK the two lists should line up)
for i in duplicates:
    if i in unique:
        pass
    else:
        counts.append(duplicates.count(i))
        unique.append(i)


In [21]:
#get the sequences for the mut cells with matching ids
seq=list(npm1wt["seq"])
duplicateseq=[]
for i in range(len(wt_ids)):
    if wt_ids[i] in duplicates:
        duplicateseq.append(seq[i])
    

In [22]:
len(duplicateseq)

2351

In [73]:
#check if mut/wt sequence is present in the sequences found as duplicates--this shows that actually the sequences do not overlap
count=0
for i in duplicateseq:
    if "CTCTGCATGG" in i:
        count+=1
print(count)

0


In [28]:
path="/home/lf2684/notebooks/" 
os.chdir(path)
cwd = os.getcwd()

#read in aml data
import scanpy as sc
sample_RNA='08H089_allsorted_cleaned.h5'
adata_RNA=sc.read_h5ad(sample_RNA)

/home/lf2684/anaconda3/lib/python3.8/site-packages/anndata/compat/__init__.py:158: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/lf2684/anaconda3/lib/python3.8/site-packages/anndata/compat/__init__.py:158: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [29]:
npm1mut = adata_RNA.obs["NPM1_mut"]
npm1wt = adata_RNA.obs["NPM1_wt"]

In [30]:
"NA" in npm1mut[0]

True

In [53]:
for i in range(npm1mut.shape[0]):
    if(npm1mut[i]=="mut"):
        if(npm1wt[i]=="wt"):
            print(temp[i])

121965741824419_08H089_Sorted_Double_P
126218845403510_08H089_2
236167189645109_08H089sorted2_1
134041207696739_08H089_Sorted_CD34
169794778130861_08H089_1
235678522362718_08H089_2
134376330312486_08H089_2
227982982690670_08H089_1
169022462585060_08H089sorted2_1
204763078450542_08H089_Sorted_CD34
162090668177827_08H089sorted2_1
204756724861676_08H089_2
201595521488732_08H089sorted2_1
200899038506228_08H089_1
131234312699308_08H089sorted2_1
230741425442019_08H089_2
195606476876132_08H089sorted2_1
135699959073116_08H089_1
131309577128237_08H089_Sorted_Double_P
130144701725979_08H089sorted2_1
200974080596892_08H089sorted2_1
235559336405796_08H089_Sorted_CD34
170137154972012_08H089_Sorted_CD34
201523581311204_08H089_2
170660067142430_08H089_2
157140435033309_08H089sorted2_1
164818809186149_08H089_Sorted_CD34
236221669755309_08H089_Sorted_Double_P
197145236794084_08H089_2
191647291231517_08H089_1
205870194444637_08H089sorted2_1
129497354660147_08H089sorted2_1
195547375515507_08H089sorted2_1

122308399844212_08H089sorted2_1
200511164180382_08H089sorted2_1
235601972062132_08H089_Sorted_CD34
232449779915574_08H089sorted2_1
165738959779756_08H089_Sorted_CD34
155971398818038_08H089_1
200899085089078_08H089_2
235214377760493_08H089_Sorted_CD34
157140435265453_08H089_Sorted_CD34
204351817275244_08H089_Sorted_CD34
227510402108204_08H089_2
231907674711923_08H089_Sorted_CD34
135700004386662_08H089sorted2_1
200426592581998_08H089_sorted
166469285108966_08H089sorted2_1
131234281244523_08H089_1
232302646614756_08H089_1
169727120227740_08H089_2
160910657083173_08H089sorted2_1
125247350521267_08H089_Sorted_CD34
239475407056227_08H089sorted2_1
196715486792942_08H089sorted2_1
205527671008550_08H089_Sorted_Double_P
239587835369371_08H089_Sorted_CD34
165946237872877_08H089sorted2_1
199934564821227_08H089_Sorted_CD34
165860292638494_08H089_1
130761060674476_08H089_1
235550861547245_08H089sorted2_1
126886731737396_08H089_2
199865991882614_08H089sorted2_1
235122023655196_08H089sorted2_1
2401368

195623356652253_08H089_2
226265786603939_08H089_Sorted_CD34
169174289497516_08H089_Sorted_Double_P
170877887866654_08H089_1
227930503825716_08H089_1
200983508007340_08H089_Sorted_CD34
162001559940006_08H089_2
157562549983478_08H089_1
204762093644708_08H089_Sorted_CD34
170265230592934_08H089_Sorted_Double_P
169158842201956_08H089sorted2_1
120726924187501_08H089_Sorted_CD34
121202323897270_08H089_1
160928757889827_08H089_Sorted_Double_P
165385745413982_08H089sorted2_1
121276814712158_08H089_1
232302646356771_08H089sorted2_1
161340107712811_08H089sorted2_1
170258473536347_08H089sorted2_1
204882352138678_08H089sorted2_1
161451938605804_08H089_Sorted_Double_P
170199713265563_08H089sorted2_1
120864497658614_08H089sorted2_1
170877934294877_08H089sorted2_1
135150470880614_08H089sorted2_1
169014052928363_08H089_sorted
191691461614325_08H089_sorted
157684165982507_08H089_Sorted_CD34
227354875444646_08H089_Sorted_CD34
226205925043501_08H089_1
240617687739230_08H089sorted2_1
126885760720230_08H089

200563789544876_08H089_Sorted_CD34
232311370725660_08H089_Sorted_CD34
196638162177267_08H089_Sorted_Double_P
240094688164707_08H089_Sorted_CD34
230799408056741_08H089sorted2_1
196019714180853_08H089_Sorted_CD34
121276814964643_08H089_sorted
165385699253556_08H089_Sorted_Double_P
226829516098267_08H089_2
232311383079286_08H089sorted2_1
201474310101220_08H089_Sorted_CD34
231890226461596_08H089_Sorted_CD34
169793589558182_08H089_Sorted_CD34
231366209189805_08H089_Sorted_Double_P
130624783235422_08H089_1
239468543429428_08H089_2
197135453801891_08H089_Sorted_CD34
169027830672172_08H089_Sorted_CD34
204360528607477_08H089_Sorted_CD34
125247350302574_08H089sorted2_1
236167209117933_08H089sorted2_1
232321033661749_08H089_Sorted_CD34
235198512778462_08H089_1
200348223618412_08H089_Sorted_CD34
240617715292014_08H089sorted2_1
170745979366110_08H089sorted2_1
232301335014638_08H089sorted2_1
197145236720566_08H089sorted2_1
164753444858780_08H089_Sorted_CD34
126680976546596_08H089sorted2_1
1662880717

160447855769948_08H089_1
227510402530677_08H089_Sorted_CD34
227903391590131_08H089_Sorted_Double_P
157486609812894_08H089_Sorted_CD34
126680930301869_08H089_Sorted_CD34
169174290319781_08H089_2
200922554067804_08H089_2
227982995831707_08H089_Sorted_CD34
170268002798389_08H089_Sorted_Double_P
130142270840101_08H089sorted2_1
169158841952179_08H089_Sorted_CD34
192718916414709_08H089_Sorted_CD34
125719977343910_08H089_2
204959662131557_08H089_2
239458074192219_08H089_Sorted_CD34
160440059022262_08H089_Sorted_Double_P
122514723985819_08H089_Sorted_CD34
126364041997022_08H089_2
236221669235612_08H089_Sorted_Double_P
130531395787572_08H089_Sorted_CD34
165946191931805_08H089sorted2_1
204756751890726_08H089_Sorted_Double_P
235550834833764_08H089_Sorted_CD34
235626667398939_08H089_Sorted_CD34
231711851567915_08H089sorted2_1
197206037359404_08H089_Sorted_CD34
197206065170676_08H089sorted2_1
231891388746141_08H089sorted2_1
231897696160694_08H089_Sorted_Double_P
170199713696603_08H089_2
23171183268

204765448522469_08H089_2
235122008635742_08H089_sorted
230592309741875_08H089sorted2_1
204224175748910_08H089_Sorted_CD34
230592295422813_08H089sorted2_1
231900127094515_08H089_Sorted_CD34
135014255056684_08H089sorted2_1
200363958790492_08H089_Sorted_Double_P
160910657612708_08H089sorted2_1
236169995348766_08H089_2
200441267926821_08H089sorted2_1
161890952133046_08H089_Sorted_Double_P
122436443100972_08H089_Sorted_Double_P
121879156681635_08H089_Sorted_CD34
125239699564851_08H089sorted2_1
160903122340773_08H089_Sorted_CD34
231770070047582_08H089_1
235548418992500_08H089_Sorted_Double_P
131175238199211_08H089_Sorted_CD34
226901468792678_08H089_Sorted_CD34
170180405283188_08H089_1
232450566843804_08H089sorted2_1
196163882871708_08H089_Sorted_Double_P
121339897403692_08H089sorted2_1
196715472178035_08H089_2
196113237437150_08H089_1
226283262502643_08H089_1
125728279451491_08H089_Sorted_CD34
122310425008030_08H089_1
164631946606957_08H089sorted2_1
165164925799267_08H089_1
200363912051115_0

In [31]:
npm1mut.shape

(37395,)

In [32]:
#create list of ids without replicate info appended
temp = adata_RNA.obs_names
adata_ids=[]
for i in temp:
    adata_ids.append(i.split("_")[0])


In [33]:
unique=[] #unique cellids
counts=[] #counts of how many times each id appears (I THINK the two lists should line up)
for i in adata_ids:
    if i in unique:
        pass
    else:
        counts.append(adata_ids.count(i))
        unique.append(i)

In [34]:
#find technical replicates from the .h5 by searching for duplicate ids (with replicate info removed)
aml_duplicates=[]
for i in range(len(counts)):
    if counts[i]>1:
        aml_duplicates.append(float(unique[i]))#,counts[i]])

In [44]:
for i in temp:
    if "197213599943084" in i:
        print(i)

197213599943084_08H089_Sorted_CD34
197213599943084_08H089_2


In [35]:
#the number of mut/wt duplicates that were also technical replicates
counter=0
for i in duplicates:
    if i in aml_duplicates:
        counter+=1
print(counter)

102


In [36]:
duplicates[0]

196043624045859.0

In [93]:
adata_ids[0]

'200899053107037'